Imputation of missing values in a time series dataset using bins of data

In [46]:
#!pip install missingpy
import os
os.chdir('/Users/sujitc/Desktop/Wearables/Signatures/Imputation_binning/21stjan')

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import EllipticEnvelope
from datetime import timedelta
from pandas import Grouper 
import impyute
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
import impyute
from missingpy import MissForest
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [48]:
# df = pd.read_csv('V1_output_data.csv')
# df['New_Time'] = df['Date'] + ' ' +df['Time']
# df['New_Time'] = pd.to_datetime(df['New_Time'])
# df=df.set_index(['New_Time'])
# df = pd.read_csv('V1_further.csv')
# # df['New_Time'] = df['Date'] + ' ' +df['Time']
# df['Datetime'] = pd.to_datetime(df['Datetime'])
# df=df.set_index(['Datetime'])
# df
df = pd.read_csv('V1_hr.csv')
# df.drop('Unnamed: 0', inplace = True, axis =1)
df['datetime'] = pd.to_datetime(df['datetime'])
df=df.set_index(['datetime'])
df

,user,heartrate
datetime,,
2020-11-01 00:00:00,V1,NaN
2020-11-01 00:01:00,V1,NaN
2020-11-01 00:02:00,V1,NaN
2020-11-01 00:03:00,V1,NaN
2020-11-01 00:04:00,V1,NaN
...,...,...
2022-06-29 23:56:00,V1,81.0
2022-06-29 23:57:00,V1,78.0
2022-06-29 23:58:00,V1,79.0


In [49]:
aug_df= df[np.logical_and(df.index>='2022-01-01 00:00:00', df.index<='2022-01-31 23:59:00')]

In [50]:
# aug_df = df[df.index.strftime("%m")=='12']
for i in range(17,18):
    binned_df1 = aug_df[(aug_df.index.strftime("%H:%M") >= '00:00') & (aug_df.index.strftime("%H:%M") <= '23:59')] 
    binned_df2 = aug_df[(aug_df.index.strftime("%H:%M") >= f'{str(i).zfill(2)}:00') & (aug_df.index.strftime("%H:%M") <= f'{str(i+1).zfill(2)}:00')]
    binned_df3 = aug_df[(aug_df.index.strftime("%H:%M") >= f'{str(i-1).zfill(2)}:30') & (aug_df.index.strftime("%H:%M") <= f'{str(i+1).zfill(2)}:30')]
    binned_df4 = aug_df[(aug_df.index.strftime("%H:%M") >= f'{str(i-1).zfill(2)}:00') & (aug_df.index.strftime("%H:%M") <= f'{str(i+2).zfill(2)}:00')]
    binned_df5 = aug_df[(aug_df.index.strftime("%H:%M") >= f'{str(i-2).zfill(2)}:30') & (aug_df.index.strftime("%H:%M") <= f'{str(i+2).zfill(2)}:30')]
    binned_df6 = aug_df[(aug_df.index.strftime("%H:%M") >= f'{str(i-2).zfill(2)}:00') & (aug_df.index.strftime("%H:%M") <= f'{str(i+3).zfill(2)}:00')]
    binned_df7 = aug_df[(aug_df.index.strftime("%H:%M") >= f'{str(i-3).zfill(2)}:30') & (aug_df.index.strftime("%H:%M") <= f'{str(i+3).zfill(2)}:30')]

    binned_df = [binned_df1, binned_df2, binned_df3, binned_df4, binned_df5, binned_df6, binned_df7]
    for dfs in binned_df:
        dfs['Hour']=dfs.index.strftime('%H').astype(float)
        dfs['Minute']=dfs.index.strftime('%M').astype(float)

    starttime=datetime.datetime(2022, 1, 21,i) 
    endtime=datetime.datetime(2022, 1, 21, i+1) 
    print(endtime)

  
    for dfs in binned_df:
        print(len(dfs))
        original=dfs.loc[np.logical_and(dfs.index >= starttime, dfs.index <= endtime)]
        dfs.loc[np.logical_and(dfs.index > starttime, dfs.index < endtime), 'heartrate'] = np.nan


    import impyute
    emerror = []
    def imputation(binned_df):
        dft_arr=binned_df.to_numpy()
        arr_em_impute=impyute.imputation.cs.em(dft_arr, loops=100)
        df_em_impute=pd.DataFrame(arr_em_impute,columns=['heartrate','Hour'],index=binned_df.index)
        return df_em_impute
    for dfs in binned_df:
        df_imputed=imputation(dfs.drop(columns=['Minute', 'user'],axis=1))
        imputed=df_imputed.loc[original.index]
        pred=imputed.heartrate
        actual=original.heartrate
        from sklearn.metrics import mean_squared_error 
        emrmse = mean_squared_error(actual, pred, squared=False)
        emrmse = "{:.2f}".format(emrmse)
        emerror.append(emrmse)
    
    ii=IterativeImputer(random_state=0)

    iierror =[]
    for dfs in binned_df:
        df_ii=ii.fit_transform(dfs.drop(columns=['Minute', 'user'],axis=1))
        df_ii=pd.DataFrame(df_ii,columns=['heartrate','Hour'],index=dfs.index)
        ii_pred=df_ii.loc[original.index]
        iipred=ii_pred.heartrate
        actual=original.heartrate
        iirmse = mean_squared_error(actual, iipred, squared=False)
        iirmse = "{:.2f}".format(iirmse)
        iierror.append(iirmse)
    from sklearn.impute import KNNImputer
    from sklearn.metrics import mean_squared_error
    knn=KNNImputer(n_neighbors=1500)

    knnerror = []
    for dfs in binned_df:
        df_knn=knn.fit_transform(dfs.drop(columns=['Minute', 'user'],axis=1))
        df_knn=pd.DataFrame(df_knn,columns=['heartrate','Hour'],index=dfs.index)
        knn_pred=df_knn.loc[np.logical_and(df_knn.index >= starttime, df_knn.index <= endtime)]
        knnpred=knn_pred.heartrate
        actual = original['heartrate']
        rmseknn = mean_squared_error(actual, knnpred, squared=False)
        rmseknn = "{:.2f}".format(rmseknn)
        knnerror.append(rmseknn)
    imputer = MissForest()
    rferror = []
    for dfs in binned_df:
        df_rf = imputer.fit_transform(dfs.drop(columns=['Minute', 'user'],axis=1))
        df_rf=pd.DataFrame(df_rf,columns=['heartrate', 'Hour'],index=dfs.index)
        rf_pred = df_rf.loc[original.index]
        rfpred = rf_pred.heartrate
        actual = original.heartrate
        rmserf = mean_squared_error(actual, rfpred, squared=False)
        rmserf = "{:.2f}".format(rmserf)
        rferror.append(rmserf)
    from sklearn.impute import SimpleImputer
    my_imputer = SimpleImputer(strategy = 'mean')

    lrerror = []
    for dfs in binned_df:
        df_simple = my_imputer.fit_transform(dfs.drop(columns=['Minute', 'user'],axis=1))
        df_simple=pd.DataFrame(df_simple,columns=['heartrate','Hour'],index=dfs.index)
        simple_pred=df_simple.loc[original.index]
        simplepred=simple_pred.heartrate
        actual=original.heartrate
        rmselr = mean_squared_error(actual, simplepred, squared=False)
        rmselr = "{:.2f}".format(rmselr)
        lrerror.append(rmselr)

    df1 = pd.DataFrame(columns = ['Bin Size','EM', 'II'] )
    df1['EM'] = emerror
    df1['II'] = iierror
    df1['KNN'] = knnerror
    df1['RF'] = rferror
    df1['SI'] = lrerror
    df1['Bin Size'] = ['Entire','1hr', '2hr', '3hr', '4hr', '5hr', '6hr']
    df1=df1.set_index(['Bin Size'])
    df1.to_csv(f'{i}-{i+1}_allday_onlymonth.csv')


    

# binned_df8 = aug_df[(aug_df.index.strftime("%H:%M") >= '01:45') & (aug_df.index.strftime("%H:%M") <= '06:45')]
# binned_df9 = aug_df[(aug_df.index.strftime("%H:%M") >= '01:15') & (aug_df.index.strftime("%H:%M") <= '07:15')]


/var/folders/x5/zgk82l8x3lj7rjsy02yylnk00000gn/T/ipykernel_4455/1989452900.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['Hour']=dfs.index.strftime('%H').astype(float)
/var/folders/x5/zgk82l8x3lj7rjsy02yylnk00000gn/T/ipykernel_4455/1989452900.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['Minute']=dfs.index.strftime('%M').astype(float)
/var/folders/x5/zgk82l8x3lj7rjsy02yylnk00000gn/T/ipykernel_4455/1989452900.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

2022-01-21 18:00:00
44640
1891
3751
5611
7471
9331
11191


/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


Iteration: 0
Iteration: 1


/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


...

/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


Iteration: 3
Iteration: 4
Iteration: 0


/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2


/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/sujitc/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
